# Runs the U-GPD Transfer Learning Model (Lapins et al, 2021) on PNR-1z data

#### Cindy Lim Shin Yee
###### Last updated: 21st July 2021
---

The U-GPD model in this notebook is developed by Lapins et al (2021).
The U-GPD model uses pre-trained weights from the GPD model (Ross et al, 2018) and uses a U-net architecture. The U-GPD underwent further fine-tuning (transfer learning) using a limited volcanic dataset from the Nabro volcano. More information and tutorials about the U-GPD model can be found here: https://github.com/sachalapins/U-GPD

Input:
The seismic dataset provided is from the PNR-1z dataset.

Output:
Event catalogue with event origin times, longitude, latitude, depth, Easting and Northing coordinates.

References:

- Lapins, S., Goitom, B., Kendall, J.M., Werner, M.J., Cashman, K.V. and Hammond, J.O., 2021. <i> A Little Data goes a Long Way: Automating Seismic Phase Arrival Picking at Nabro Volcano with Transfer Learning</i>. Journal of Geophysical Research: Solid Earth, p.e2021JB021910. https://doi.org/10.1002/essoar.10506323.1

- Ross, Z.E., Meier, M.A., Hauksson, E. and Heaton, T.H., 2018. <i> Generalized seismic phase detection with deep learning. </i> Bulletin of the Seismological Society of America, 108(5A), pp.2894-2901. https://doi.org/10.1785/0120180080

## 1. Installing libraries/toolboxes

[1] Obspy:
**IMPORTANT:** You must restart runtime after this step and then run notebook again from the beginning, otherwise obspy will not read or download data correctly.

[2] Pyproj:
Library for transforming coordinates and cartographic projections.

In [ ]:
!pip install --upgrade obspy

In [ ]:
!pip install pyproj

In [ ]:
!pip install h5py==2.10.0

## 2. Load modules etc

Make sure Hardware Accelerator is set to use GPU (click on 'Runtime' menu at top, then 'Change runtime type', then make sure GPU is selected from drop down menu under 'Hardware accelerator').

Then, let's load required modules and define some functions.

In [ ]:
# Original GPD model was produced using TensorFlow (TF) version 1, so we need to tell Google Colab to use v1 (comment out this step if using conda environment above)
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# Import modules
import string
import time
import argparse as ap
import sys
import os
import glob

import numpy as np
import pandas as pd
import obspy.core as oc
from obspy.core import read
from obspy.signal.trigger import trigger_onset
import obspy.signal
import obspy.signal.filter as obs
from obspy.core.utcdatetime import UTCDateTime

import matplotlib.pyplot as plt

import math # for pi, sqrt, etc

import tensorflow as tf
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K
from keras import losses
from keras.legacy import interfaces

import h5py
import json

import random

import scipy.stats as stats
import scipy.signal as signal

from datetime import datetime
from scipy.interpolate import InterpolatedUnivariateSpline
import dateutil

# for location
import pyproj

Using TensorFlow backend.


In [ ]:
# Adam optimiser with variable learning rates (applied through a learning rate multiplier)
# This was used during model training to set a learning rate 'multiplier' for different layers (so we can have lower learning rate for GPD layers)
# It needs to be defined so we can load our trained model

# Adapted from: https://erikbrorson.github.io/2018/04/30/Adam-with-learning-rate-multipliers/


class Adam_lr_mult(Optimizer):
    """Adam optimizer.
    Adam optimizer, with learning rate multipliers built on Keras implementation
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        
    AUTHOR: Erik Brorson
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False,
                 multipliers=None, debug_verbose=False,**kwargs):
        super(Adam_lr_mult, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self._lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.multipliers = multipliers
        self.debug_verbose = debug_verbose

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self._lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):

            # Learning rate multipliers
            if self.multipliers:
                multiplier = [mult for mult in self.multipliers if mult in p.name]
            else:
                multiplier = None
            if multiplier:
                new_lr_t = lr_t * self.multipliers[multiplier[0]]
                if self.debug_verbose:
                    print('Setting {} to learning rate {}'.format(multiplier[0], new_lr_t))
                    print(K.get_value(new_lr_t))
            else:
                new_lr_t = lr_t
                if self.debug_verbose:
                    print('No change in learning rate {}'.format(p.name))
                    print(K.get_value(new_lr_t))
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - new_lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - new_lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self._lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad,
                  'multipliers':self.multipliers}
        base_config = super(Adam_lr_mult, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# Define function to produce sliding windows of signal (from original GPD GitHub repo https://github.com/interseismic/generalized-phase-detection/blob/master/gpd_predict.py)

def sliding_window(data, size, stepsize=1, padded=False, axis=-1, copy=True):
    """
    Calculate a sliding window over a signal
    Parameters
    ----------
    data : numpy array
        The array to be slided over.
    size : int
        The sliding window size
    stepsize : int
        The sliding window stepsize. Defaults to 1.
    axis : int
        The axis to slide over. Defaults to the last axis.
    copy : bool
        Return strided array as copy to avoid sideffects when manipulating the
        output array.
    Returns
    -------
    data : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window.
    Notes
    -----
    - Be wary of setting `copy` to `False` as undesired sideffects with the
      output values may occurr.
    Examples
    --------
    >>> a = numpy.array([1, 2, 3, 4, 5])
    >>> sliding_window(a, size=3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5]])
    >>> sliding_window(a, size=3, stepsize=2)
    array([[1, 2, 3],
           [3, 4, 5]])
    See Also
    --------
    pieces : Calculate number of pieces available by sliding
    """
    if axis >= data.ndim:
        raise ValueError(
            "Axis value out of range"
        )

    if stepsize < 1:
        raise ValueError(
            "Stepsize may not be zero or negative"
        )

    if size > data.shape[axis]:
        raise ValueError(
            "Sliding window size may not exceed size of selected axis"
        )

    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / stepsize - size / stepsize + 1).astype(int)
    shape.append(size)

    strides = list(data.strides)
    strides[axis] *= stepsize
    strides.append(data.strides[axis])

    strided = np.lib.stride_tricks.as_strided(
        data, shape=shape, strides=strides
    )

    if copy:
        return strided.copy()
    else:
        return strided?

In [ ]:
# Defining file reading function (from Alan Baird, October 2019)
def readPNR(fname,**kwargs):
    """Wrapper function for reading PNR segy data which sets headers for
    stations and channels automatically"""
    st = read(fname,**kwargs)
    # populate trace headers for network, station name, channel, and distance (depth, needed to easily produce section plots)
    for i, trace in enumerate(st):
        trace.stats.network = "PNR"
        trace.stats.station = "{:0>3d}".format(trace.stats.segy.trace_header.geophone_group_number_of_last_trace)
        trace.stats.channel = 'BS'+str(i%3+1)
        trace.stats.distance = trace.stats.segy.trace_header.receiver_group_elevation/1000
    return st

# Defining trace rotating function
def readPNR_ENZ(fname,**kwargs):
    """Wrapper function for reading PNR segy data and reorienting it into ENZ
    components witgh appropriate headers.

    Note: Requires that the stns dataframe has been read in correctly"""
    st = readPNR(fname,**kwargs)

    # Create a copy of the traces to be rotated
    strot=st.copy()

    # Loop through stations, construct rotation matrices and apply to apply to traces
    for i, row in stns.iterrows():
        azi=np.radians(row['azi'])
        inc=np.radians(row['inc'])
        rot=np.radians(row['rgb_orient'])

        # Create rotation matrices
        R1=np.array([[np.sin(azi), -np.cos(azi), 0],
                     [np.cos(azi),  np.sin(azi), 0],
                     [          0,            0, 1]])
        R2=np.array([[np.cos(inc), 0, -np.sin(inc)],
                     [          0, 1,            0],
                     [np.sin(inc), 0,  np.cos(inc)]])
        R3=np.array([[ np.cos(rot), np.sin(rot), 0],
                     [-np.sin(rot), np.cos(rot), 0],
                     [           0,           0, 1]])

        # Select traces from appropriate station
        stnm = "{:0>3d}".format(int(row['name']))
        sttmp = strot.select(station=stnm)

        # instrument coordinates are left-handed Hx,Hy,Vz, need to flip polarization of Hy to make it right handed
        tmparrays = np.stack((sttmp[0].data,-sttmp[1].data,sttmp[2].data))

        # Apply the rotation
        rotated = R1 @ R2 @ R3 @ tmparrays

        # Set the data of the traces
        sttmp[0].data=rotated[0,:]
        sttmp[1].data=rotated[1,:]
        sttmp[2].data=rotated[2,:]

        # Rename the channels
        sttmp[0].stats.channel = 'BSE'
        sttmp[1].stats.channel = 'BSN'
        sttmp[2].stats.channel = 'BSZ'
    return strot

## 3. Download and load trained U-GPD Transfer Learning model

Next let's clone the repo for our trained U-GPD Transfer Learning model and load our trained model with weights from the training epoch that gave lowest validation loss.

In [ ]:
!git clone https://github.com/sachalapins/U-GPD.git

fatal: destination path 'U-GPD' already exists and is not an empty directory.


In [ ]:
# Load model and weights
# If there is a warning/exception: reinstall h5py == 2.10.0
model = keras.models.load_model('./U-GPD/model/best/model.h5',custom_objects={'Adam_lr_mult': Adam_lr_mult})
print("*** 1 of 2: MODEL LOADED FROM DISK ***")

model.load_weights('./U-GPD/model/best/weights_lowest_loss.hdf5') # Load in weights from training epoch with lowest validation loss

print ("*** 2 of 2:BEST TRAINING WEIGHTS LOADED ***")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

*** 1 of 2: MODEL LOADED FROM DISK ***
*** 2 of 2:BEST TRAINING WEIGHTS LOADED ***


## 4. Download the PNR catalogue from the Coalescence Microseismic Mapping (CMM) method.

We chose an example hour of the PNR data (11 December 2018, 9-10am) to run the model on.

Note: can replace this with your own data.

In [ ]:
!git clone https://github.com/cl16908/PNR_run.git

# Define some paths
drive = './PNR_run/'

# Read the station orientations for trace rotations
# read in event catalogue as a dataframe (note that these don't have traveltime picks)
df_cat = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_FullCatalogue.dat',
                    delim_whitespace=True)

# create a new datetime column with the origin times of the events
df_cat['datetime']=df_cat['T'].apply(UTCDateTime) + df_cat['QC_LOC_T0']

# well path 1
wp1 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Wellpath.dat', delim_whitespace=True)

# well path 2
wp2 = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Wellpath.dat', delim_whitespace=True)

# location of the injection sleeves of pnr1 and 2
pnr1_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stages.dat', delim_whitespace=True)
pnr2_stgs = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-2_Stages.dat', delim_whitespace=True)

# station locations and "orientations"
stns = pd.read_csv(drive + '/Catalog/Event_and_Station/PNR-1z_Stations_orient.dat', delim_whitespace=True)


# Functions to return well inclination and azimuth from measured depth
md2inc=InterpolatedUnivariateSpline(wp2['MD'],wp2['Inc'])
md2az=InterpolatedUnivariateSpline(wp2['MD'],wp2['Az'])


# Find measured depth for each station by root finding
MDs=[]
for stdp in stns['stz']:
    MDs.append(InterpolatedUnivariateSpline(wp2['MD'],wp2['TVD_MSL']-stdp).roots()[0])

# assign station measured depth and compute station azimuth and inclination
stns['MD'] = MDs
stns['azi']=stns['MD'].apply(md2az)
stns['inc']=stns['MD'].apply(md2inc)

fatal: destination path 'PNR_run' already exists and is not an empty directory.


In [ ]:
# Reading filenames of the segy files

this_folder = '181211'
hour = '09'
filenames1 = int('20'+this_folder + hour)
filenames = drive + '/data/' + this_folder[0:6] + '/' + str(filenames1) + '*.segy'
fhour = glob.glob(filenames)
fhour.sort()

In [ ]:
# ** RESTART NOTEBOOK AND REINSTALL OBSPY IF THIS CELL DOES NOT RUN **
# Load waveforms into obspy stream - takes about 1 to 2 minutes

# Pre-allocate empty [n_station x n_samples x 3 component] matrix space;
# [24 x (16 seconds*2000Hz*len(filenames)) x 3]
a = np.zeros((24,2000*16*len(fhour),3))

try:
  # loop that reads in the data per filename
  for j in np.arange(0,len(fhour)):
      sthour = readPNR_ENZ(fhour[j],unpack_trace_headers = True)
      for i in np.arange(0,len(sthour)):
          a[math.floor(i/3),np.arange(j*2000*16,(j+1)*2000*16),(i%3)] = sthour[i].data # station number, data number, channel number
except:
  !pip uninstall -y obspy # then restart from beginning
  print("***Please restart notebook from beginning***")


In [ ]:
# Stream naming

# Define first file name
name = fhour[0]
# set starttime
for i in np.arange(0,len(sthour)):
    sthour[i].stats.starttime = UTCDateTime(int(name[-23:-19]), int(name[-19:-17]), int(name[-17:-15]), int(name[-15:-13]), int(name[-13:-11]), int(name[-11:-9]))

In [ ]:
# Preprocessing the data matrix
# Detrend and filter data (and resample) as per inputs above
for i in np.arange(0,(a.shape[0])-1):
    a[i,:,0] = obspy.signal.detrend.simple(a[i,:,0])
    a[i,:,1] = obspy.signal.detrend.simple(a[i,:,1])
    a[i,:,2] = obspy.signal.detrend.simple(a[i,:,2])

freq_max = 50  # using a 50 Hz highpass frequency

# filter data matrix
for i in np.arange(0,(a.shape[0])-1):
    a[i,:,0] = obs.highpass(a[i,:,0],freq=freq_max,df=sthour[0].stats.sampling_rate)
    a[i,:,1] = obs.highpass(a[i,:,1], freq=freq_max, df=sthour[0].stats.sampling_rate)
    a[i,:,2] = obs.highpass(a[i,:,2], freq=freq_max, df=sthour[0].stats.sampling_rate)


## 6. Define some parameters for sliding window and phase arrival trigger

In [ ]:
# Model prediction parameters
shift_size = 200 # Overlap (in samples) for sliding window
batch_size = 100 # No. of windows to process at a time

# Trigger parameters
p_thresh = 0.4 # Trigger on threshold for P-waves
s_thresh = 0.4 # Trigger on threshold for S-waves
trig_off = 0.2 # Trigerr off threshold for both phase arrivals

## 7. Run model using sliding window on continuous data

In this step, we loop through each available station, dividing the data from each instrument channel into overlapping 400-sample windows (with 200-sample overlap) and then run our model on these windows.

We keep only the middle 200 samples from each window to avoid poor predictions at the window edges.

We then apply a simple trigger algorithm in obspy to a concatenated array of all these window predictions (one long prediction trace with same sample rate as the input data). Any P- or S-wave predictions that exceed the trigger thresholds given above (`p_thresh` and `s_thresh`) will be added to the dataframe `df` of phase arrival pick times, along with their 'probabilities' (a measure of model confidence in that phase arrival classification).

In [ ]:
# MODEL RUNNING - takes about 3 minutes for one PNR hour (2GB)
# Dataframe to store picks from trigger
df = pd.DataFrame(columns=['time', 'sta', 'pha', 'prob'])

# Loop through list of available stations
before = datetime.now() # Get time before and after to see how long this takes
for s in np.arange(0, a.shape[0]):
    
    # Expand overlapping windows using sliding_window function
    sliding_E = sliding_window(a[s,:,0], 400, stepsize=shift_size) # Slide window across first component
    tr_win = np.zeros((sliding_E.shape[0], 400, 3)) # 3D array (of zeros)
    tr_win[:,:,0] = sliding_E # Add to 3D array
    sliding_E = None # clear some memory
    
    sliding_N = sliding_window(a[s,:,1], 400, stepsize=shift_size) # Slide window across second component
    tr_win[:,:,1] = sliding_N # Add to 3D array
    sliding_N = None # clear some memory
    
    sliding_Z = sliding_window(a[s,:,2], 400, stepsize=shift_size) # Slide window across third component
    tr_win[:,:,2] = sliding_Z # Add to 3D array
    sliding_Z = None # clear some memory
    
    tr_win = signal.detrend(tr_win, axis=1) # Detrend 
    tr_win = tr_win / np.max(np.abs(tr_win), axis=(1,2))[:,None,None] # Normalize between 0 and 1 (divide by max across 3 components for each window)
    
    # Model prediction step...
    ts = model.predict(tr_win, verbose=True, batch_size=batch_size)
    
    # Merge overlapping predictions:
    n_steps_per_win = 400/shift_size # No. of steps per window (i.e. how many overlapping windows)
    class_trace = np.zeros((len(a[s,:,0]), int(3))) # Create prediction traces to be same size as input data
    for i in np.arange(0, ts.shape[0]): # For each window
        class_trace[((i * shift_size) + 100):((i * shift_size) + 300), :] += ts[i, 100:300, :] # Keep middle 200 predictions

    # Use obspy trigger to find predictions above threshold (i.e. p_thresh and s_thresh)
    # P-wave picks (trigger):
    trigs = trigger_onset(class_trace[:,0], p_thresh, trig_off) # Look for triggers (prediction trace exceeding p_thresh)
    p_picks = []
    p_probs = []
    for trig in trigs:
        if trig[1] == trig[0]:
            continue # If trigger on and off times are the same then ignore
        pick = np.argmax(class_trace[trig[0]:trig[1], 0]) + trig[0] # Use argmax between trigger on and off times as pick time
        stamp_pick = sthour[0].stats.starttime + (pick * sthour[0].stats.delta) # Pick time as UTCDateTime
        p_picks.append(stamp_pick) # Append time to list of P pick times
        p_probs.append(class_trace[pick, 0]) # Append prob to list of P pick probabilities
    # Add picks to df for this day            
    if len(p_picks) > 0:
        df_add = pd.DataFrame(p_picks, columns=['time'])
        df_add['pha'] = 'p'
        if (s+1) > 9:
          df_add['sta'] = "0" + str(s+1)
        else:
          df_add['sta'] = "00" + str(s+1)
        df_add['prob'] = p_probs
        df = pd.concat([df, df_add])

    # Repeat above for S-wave picks and s_thresh:
    trigs = trigger_onset(class_trace[:,1], s_thresh, trig_off)
    s_picks = []
    s_probs = []
    for trig in trigs:
        if trig[1] == trig[0]:
            continue
        pick = np.argmax(class_trace[trig[0]:trig[1], 1]) + trig[0]
        stamp_pick = sthour[0].stats.starttime + (pick * sthour[0].stats.delta)
        s_picks.append(stamp_pick)
        s_probs.append(class_trace[pick, 1])
    # Add picks to df for this day
    if len(s_picks) > 0:
        df_add = pd.DataFrame(s_picks, columns=['time'])
        df_add['pha'] = 's'
        if (s+1) > 9:
          df_add['sta'] = "0" + str(s+1)
        else:
          df_add['sta'] = "00" + str(s+1)
        df_add['prob'] = s_probs
        df = pd.concat([df, df_add])


df = df.sort_values(by='time').reset_index(drop=True) # Sort df by pick times
#df.to_pickle("trigs.pkl") # Write df to pkl file - uncomment if you want to save your model picks

after = datetime.now()
time_taken = after-before
print("1 hour data from " + str(a.shape[0]) + " stations processed in " + str(time_taken.seconds) + "." + str(time_taken.microseconds) + " seconds.")    

# # uncomment to save dataframe
# df.to_csv(drive + 'df_UGPD.csv')

35999/35999 [==============================] - 4s 110us/step
1 hour data from 24 stations processed in 179.344005 seconds.


In [ ]:
df

,time,sta,pha,prob
0,2018-12-11T09:00:04.043000Z,015,s,0.401806
1,2018-12-11T09:00:14.425000Z,020,s,0.546195
2,2018-12-11T09:00:16.339500Z,015,s,0.567049
3,2018-12-11T09:00:23.274000Z,007,s,0.829118
4,2018-12-11T09:00:27.625000Z,007,s,0.995186
...,...,...,...,...
71983,2018-12-11T10:00:01.526000Z,023,p,0.702678
71984,2018-12-11T10:00:01.574000Z,023,s,0.834483
71985,2018-12-11T10:00:01.582000Z,022,s,0.659105
71986,2018-12-11T10:00:01.585500Z,022,s,0.604628


## 8. Associate predicted phase arrivals for event location

### Grouping Ps and Ss into events

We first group P-wave arrivals into 0.2-second 'bins'.
0.2 seconds should cover the largest arrival time difference between all 24 stations (i.e. 0.2 seconds > maximum theoretical travel time between any two stations).
We then group the S-waves into similar 0.2-second bins

In [ ]:
###############################
# Grouping Ps - 1 to 2 minutes
###############################
# Dataframe with only Ps
df_P = df[df['pha']=='p']
df_P = df_P.reset_index(drop=True)

# convert all string 'time' entries to UTCDateTime

for i in np.arange(0,len(df_P)):
    df_P['time'].values[i] = UTCDateTime(df_P['time'].values[i])

timewin = 0.2 # time window for 'event' groups
count = 0
ngroup = 1 # set group number to 0
group = pd.DataFrame(columns=['time','sta','pha','prob']) # set an empty 'group' dataframe
j = 0 # will be the 'header' event time (earliest time in the event group for naming)
Pevs = pd.DataFrame(columns=['time','sta','pha','prob','group_no'])

# loop through the whole df_P (2 mins 30s/ 1MIN 10S)
for i in np.arange(0,len(df_P)):
    # if the timestamp is <= the upper time limit, add details to 'group_add' df to be concatenated to 'group' df   
    if (df_P['time'].values[i] <= df_P['time'].values[j]+timewin):
        count += 1
        group_add = pd.DataFrame(index= np.arange(0,1),columns=['time','sta','pha','prob'])
        group_add['time'] = df_P['time'].values[i] 
        group_add['sta'] = df_P['sta'].values[i]
        group_add['pha'] = df_P['pha'].values[i]
        group_add['prob'] = df_P['prob'].values[i]
        group = pd.concat([group,group_add])   

    # if time is > the upper time limit, it is a new event group
    else:
      # if the group length is >= 4, add to PSevs
        if len(group)>= 4:
          group['group_no'] = [ngroup] * len(group)
          Pevs = pd.concat([Pevs, group])
          ngroup += 1
        else:
          pass
      
        # j index will be the first/header event time name, increase by i for the next header event time
        j = i

        # make new group, with the new header event as first entry 
        group = pd.DataFrame(columns=['time','sta','pha','prob']) # empty df
        group_add['time'] = df_P['time'].values[i] 
        group_add['sta'] = df_P['sta'].values[i]
        group_add['pha'] = df_P['pha'].values[i]
        group_add['prob'] = df_P['prob'].values[i]
        group = pd.concat([group,group_add])

# out of loop, save the last group
j = i
if len(group)>= 4:
  group['group_no'] = [ngroup] * len(group)
  Pevs = pd.concat([Pevs, group])
  ngroup += 1
 

In [ ]:
######################################
# Grouping Ss - takes about 1 to 2 minutes
######################################

df_S = df[df['pha']=='s']
df_S = df_S.reset_index(drop=True)

# converting all string entries into UTCDateTime
for i in np.arange(0,len(df_S)):
    df_S['time'].values[i] = UTCDateTime(df_S['time'].values[i])

timewin = 0.2
count = 0
ngroup = 1
group = pd.DataFrame(columns=['time','sta','pha','prob'])
j = 0
Sevs = pd.DataFrame(columns=['time','sta','pha','prob','group_no'])

# loop through the whole df_S (2 mins 30s/ 1MIN45S)
for i in np.arange(0,len(df_S)):
    # if the timestamp is <= the upper time limit, add details to 'group_add' df to be concatenated to 'group' df   
    if (df_S['time'].values[i] <= df_S['time'].values[j]+timewin):
        count += 1
        group_add = pd.DataFrame(index= np.arange(0,1),columns=['time','sta','pha','prob'])
        group_add['time'] = df_S['time'].values[i] 
        group_add['sta'] = df_S['sta'].values[i]
        group_add['pha'] = df_S['pha'].values[i]
        group_add['prob'] = df_S['prob'].values[i]
        group = pd.concat([group,group_add])   

    # if time is > the upper time limit, it is a new event group
    else:
      # if the group length is >= 4, add to PSevs
        if len(group)>= 4:
          group['group_no'] = [ngroup] * len(group)
          Sevs = pd.concat([Sevs, group])
          ngroup += 1
        else:
          pass
      
        # j index will be the first/header event time name, increase by i for the next header event time
        j = i

        # make new group, with the new header event as first entry 
        group = pd.DataFrame(columns=['time','sta','pha','prob']) # empty df
        group_add['time'] = df_S['time'].values[i] 
        group_add['sta'] = df_S['sta'].values[i]
        group_add['pha'] = df_S['pha'].values[i]
        group_add['prob'] = df_S['prob'].values[i]
        group = pd.concat([group,group_add])

# out of loop, save the last group
j = i
if len(group)>= 4:
  group['group_no'] = [ngroup] * len(group)
  Sevs = pd.concat([Sevs, group])
  ngroup += 1


### Phase Association

Associating the S-wave picks to P-wave picks using a 0.13-second window.

Only associating S-wave picks to P-wave picks if:
(Ptime <= Stime <= Ptime + 0.13)

In [ ]:
# WORKS -- takes a long time ~35 min
# Associating S to the Ps
# if  Ptime <= Stime <= Ptime + 0.13

PSevs = Pevs.copy()
PSevs['stime'] = np.nan * len(PSevs)
Stime = []
# add Pevs['prob_s']
PSevs['prob_s'] = np.nan * len(PSevs)
# PSevs reset index
PSevs = PSevs.reset_index(drop=True)

# for all P phase entries, search through the S arrival picks
for j in np.arange(0,len(PSevs)):
    Stime = []
    # same station in Sevs dataframe
    samesta_df = Sevs[Sevs['sta'] == PSevs['sta'].iloc[j]]
    samesta_df = samesta_df.reset_index(drop=True)

    for i in np.arange(0,len(samesta_df)):
        # if within time window, add to Stime matrix
      if (samesta_df['time'].iloc[i] <= PSevs['time'].iloc[j] + 0.13) and (samesta_df['time'].iloc[i] >= PSevs['time'].iloc[j]):
        Stime.append(samesta_df['time'].iloc[i])
            
        # if S picks have multiple times, take the higher prob_S
        if len(Stime)>1:
                
          # if new Sevs prob > prob already set, overwrite:
          if Sevs['prob'].iloc[i] > PSevs['prob_s'].iloc[j]:
              PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
              PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]
              # print('added! x2')
              
          else:
              pass
        # else add to PSevs dataframe
        else:
            
            PSevs['stime'].iloc[j] = samesta_df['time'].iloc[i]
            PSevs['prob_s'].iloc[j] = samesta_df['prob'].iloc[i]

    else:
        pass          

PSevs['pha_s'] = ['s'] * len(PSevs)

# resetting index
PSevs = PSevs.reset_index(drop=True)

# # Saving
# PSevs.to_csv(drive+'/testblanketPS.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
# add a null space after every Stime
idxnan = []
for i in np.arange(0,len(PSevs)):
    if pd.isnull(PSevs['stime'].iloc[i]):
        pass
    else:
        idxnan.append(i)

for i in np.arange(0,len(idxnan)):
    idxnan[i] = idxnan[i] + .5

idxnew = PSevs.index.union(idxnan)[:-1]
PSevs = PSevs.reindex(idxnew).reset_index(drop=True)

# if time value is nan, fill in with details from before
for i in np.arange(1,len(PSevs)):
    
    if pd.isnull(PSevs['time'].iloc[i]):
        PSevs['time'].values[i] = PSevs['stime'].values[i-1]
        PSevs['sta'].values[i] = PSevs['sta'].values[i-1]
        PSevs['pha'].values[i] = PSevs['pha_s'].values[i-1]
        PSevs['prob'].values[i] = PSevs['prob_s'].values[i-1]
        PSevs['group_no'].values[i] = PSevs['group_no'].values[i-1]
    else:
        pass

# empty space after every group no
idx = PSevs.index.union(PSevs.index[PSevs['group_no'].shift(-1).ne(PSevs['group_no'])] + .5)[:-1]

PSevs = PSevs.reindex(idx).reset_index(drop=True)

# dropping all unnecessary columns
PSevs = PSevs.drop(columns=['stime','prob_s','pha_s','group_no'])

cols = ['sta','pha','prob','time']

PSevs = PSevs[cols]

In [ ]:
# # Uncomment to save
# PSevs.to_csv(drive+'/testblanketPS.csv')
# # load PSevs
# PSevs = pd.read_csv(drive+'/testblanketPS.csv',index_col=0)

In [ ]:
PSevs

,sta,pha,prob,time
0,016,p,0.525291,2018-12-11T09:01:05.918000Z
1,016,s,0.988732,2018-12-11T09:01:05.959500Z
2,017,p,0.832068,2018-12-11T09:01:05.941000Z
3,017,s,0.917323,2018-12-11T09:01:05.986500Z
4,017,p,0.566624,2018-12-11T09:01:05.960500Z
...,...,...,...,...
52734,022,p,0.976761,2018-12-11T10:00:00.586500Z
52735,022,s,0.893371,2018-12-11T10:00:00.634000Z
52736,021,p,0.908123,2018-12-11T10:00:00.590000Z
52737,021,s,0.960965,2018-12-11T10:00:00.645000Z


## 9. Event Location

We use NLLoc to obtain event locations from the grouped and associated phase picks. To get an event catalogue along with their locations, we use NLLoc to estimate origin times for each event.


1. Observation File Maker

  Making the observation file (.hpf file) in the NLLOC_OBS format.

2. NLLoc

  Running the NLLoc algorithm on the observation file.

3. NLLoc output to Event Catalogue

  Transforming the output files from NLLoc to an event catalogue with origin times, latitude, longitude, depth, Easting and Northing.

In [ ]:
##################################
# OBSERVATION FILE MAKER
##################################

# new dataframe for nlloc
PSobs = pd.DataFrame(columns=['sta','instrument','comp','Ponset','phase','firstmotion','date(yyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period','priorwt'])

# convert all UTCDateTime times to string
for i in np.arange(0,len(PSevs)):

  PSevs['time'].values[i] = str(PSevs['time'].values[i])

  if isinstance(PSevs['time'].values[i],str):
      PSevs['time'].values[i] = PSevs['time'].values[i][2:4] + PSevs['time'].values[i][5:7] + PSevs['time'].values[i][8:10] + PSevs['time'].values[i][11:13] + PSevs['time'].values[i][14:16] + PSevs['time'].values[i][17:25]
      # PSevs['time'].values[i] = '{:07.4f}'.format(PSevs['time'].values[i])
  else:
      pass

# making sure each station number is at least 2 characters

for i in np.arange(0,len(PSevs)):
    if (not isinstance(PSevs['sta'].iloc[i],str)) and (PSevs['sta'].iloc[i] < 10):
        PSevs['sta'].iloc[i] = '0' + str(int(PSevs['sta'].iloc[i]))
        
    elif (not isinstance(PSevs['sta'].iloc[i],str)) and (math.isnan(PSevs['sta'].iloc[i])):
        pass
    
    else:
        PSevs['sta'].iloc[i] = str(int(PSevs['sta'].iloc[i]))

PSevs = PSevs.reset_index(drop=True)

# setting station name to PR + station number

for i in np.arange(len(PSevs)):
    
    if isinstance(PSevs['sta'].iloc[i],str):
      if int(PSevs['sta'].iloc[i]) < 10:
        PSevs['sta'].iloc[i] = 'PR0' + PSevs['sta'].iloc[i] + "I" + "P"
      else:
        PSevs['sta'].iloc[i] = 'PR' + PSevs['sta'].iloc[i] + "I" + "P"
    
    else:
        pass

PSobs['sta'] = PSevs['sta']
PSobs['date(yyyymmdd)'] = PSevs['time']
PSobs['hhmm'] = PSevs['time']
PSobs['ss'] = PSevs['time']

# defining station, instrument, comp, Ponset, phase, firstmotion, date, err, coda_duration, amplitude and period
for i in np.arange(0,len(PSevs)):
    
    if PSobs['sta'].values[i] == str(PSobs['sta'].values[i]):

        PSobs['sta'].values[i] = PSevs['sta'].values[i][0:4]
        
        PSobs['instrument'].values[i] = '?'
        PSobs['comp'].values[i] = '?'
        PSobs['Ponset'].values[i] = '?'
        PSobs['phase'].values[i] = PSevs['pha'].values[i]
        PSobs['firstmotion'].values[i] = '?'
        PSobs['date(yyyymmdd)'].values[i] = '20' + PSevs['time'].values[i][0:4] + PSevs['time'].values[i][4:6]
        PSobs['hhmm'].values[i] = PSevs['time'].values[i][6:10]
        PSobs['ss'].values[i] = PSevs['time'].values[i][10:]
        PSobs['err'].values[i] = 'GAU'
        PSobs['errmag'].values[i] = '?'
        PSobs['coda_duration'].values[i] = -1.00e+00
        PSobs['amplitude'].values[i] =-1.00e+00
        PSobs['period'].values[i] =-1.00e+00  
        
    else:
        pass
    
# defining probability ranking and errmag columns
PSobs['prob'] = PSevs['prob']
PSobs['prob_rank']= np.arange(0,len(PSobs))
prob_rank = []
errmag = []
for i in np.arange(0,len(PSevs)):

    if PSobs['prob'].values[i] >= 0.85:
        prob_rank.append(0)
        errmag.append(5*1/2000)

    elif PSobs['prob'].values[i] >=0.7:
        prob_rank.append(1)
        errmag.append(10*1/2000)
        
    elif PSobs['prob'].values[i] >= 0.6:
        prob_rank.append(2)
        errmag.append(20*1/2000)

    elif PSobs['prob'].values[i] >= 0.5:
        prob_rank.append(3)
        errmag.append(50*1/2000)
        
    elif PSobs['prob'].values[i] < 0.5:
        prob_rank.append(4)
        errmag.append(99999.9)
        
    elif math.isnan(PSobs['prob'].values[i]):
        prob_rank.append(np.NaN)
        errmag.append(np.NaN)
PSobs['prob_rank'] = prob_rank
PSobs['errmag'] = errmag

for i in np.arange(0,len(PSobs)):
   if not isinstance(PSobs['sta'].iloc[i],str):
      PSobs['date(yyyymmdd)'].iloc[i] = np.nan
      PSobs['hhmm'].iloc[i] = np.nan
      PSobs['ss'].iloc[i] = np.nan
   else:
      pass

print('Saving hpf file.')
PSobs[['sta','instrument','comp','Ponset','phase','firstmotion','date(yyyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period']].to_csv(drive +  '/testNLLOC_OBS.hpf', header=None, index=None, sep=' ')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Saving hpf file.


In [ ]:
PSobs[['sta','instrument','comp','Ponset','phase','firstmotion','date(yyyymmdd)','hhmm','ss','err','errmag','coda_duration','amplitude','period']]

,sta,instrument,comp,Ponset,phase,firstmotion,date(yyyymmdd),hhmm,ss,err,errmag,coda_duration,amplitude,period
0,PR16,?,?,?,p,?,20181211,0901,05.91800,GAU,0.0250,-1,-1,-1
1,PR16,?,?,?,s,?,20181211,0901,05.95950,GAU,0.0025,-1,-1,-1
2,PR17,?,?,?,p,?,20181211,0901,05.94100,GAU,0.0050,-1,-1,-1
3,PR17,?,?,?,s,?,20181211,0901,05.98650,GAU,0.0025,-1,-1,-1
4,PR17,?,?,?,p,?,20181211,0901,05.96050,GAU,0.0250,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52734,PR22,?,?,?,p,?,20181211,1000,00.58650,GAU,0.0025,-1,-1,-1
52735,PR22,?,?,?,s,?,20181211,1000,00.63400,GAU,0.0025,-1,-1,-1
52736,PR21,?,?,?,p,?,20181211,1000,00.59000,GAU,0.0025,-1,-1,-1
52737,PR21,?,?,?,s,?,20181211,1000,00.64500,GAU,0.0025,-1,-1,-1


In [ ]:
######################
# NLLOC 
######################

# download NLLoc
!wget http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz; tar -zxf NLL7.00_src.tgz; cd src; make distrib

# make directories

!mkdir ./NLLoc
!mkdir ./NLLoc/picks
!mkdir ./NLLoc/model
!mkdir ./NLLoc/time
!mkdir ./NLLoc/loc

--2021-07-21 17:26:37--  http://alomax.free.fr/nlloc/soft7.00/tar/NLL7.00_src.tgz
Resolving alomax.free.fr (alomax.free.fr)... 212.27.63.116
Connecting to alomax.free.fr (alomax.free.fr)|212.27.63.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407847 (398K) [application/x-gzip]
Saving to: ‘NLL7.00_src.tgz.1’

NLL7.00_src.tgz.1   100%[===================>] 398.29K   581KB/s    in 0.7s    

2021-07-21 17:26:38 (581 KB/s) - ‘NLL7.00_src.tgz.1’ saved [407847/407847]

make: Nothing to be done for 'distrib'.
mkdir: cannot create directory ‘./NLLoc’: File exists
mkdir: cannot create directory ‘./NLLoc/picks’: File exists
mkdir: cannot create directory ‘./NLLoc/model’: File exists
mkdir: cannot create directory ‘./NLLoc/time’: File exists
mkdir: cannot create directory ‘./NLLoc/loc’: File exists


In [ ]:
# blanketPScindy_nll.in
! ./src/Vel2Grid ./PNR_run/blanketPScindy_nll.in

Vel2Grid (NonLinLoc v7.00.00 27Oct2017) 
CONTROL:  MessageFlag: 3  RandomNumSeed: 54321
TRANSFORM  SIMPLE LatOrig 53.775770  LongOrig -2.987780  RotCW 0.000000
Vel2Grid files:  Output: ./NLLoc/model/3km0.025.*
Vel2Grid wave type:  P
Vel2Grid wave type:  S
GRID: {x, y, z}
  Num: {121, 121, 121}
  Orig: {0, 0, 0}
  LenSide: {0.025, 0.025, 0.025}
  Type: SLOW_LEN
Creating model grid files: ./NLLoc/model/3km0.025.P.mod.*
Creating model grid files: ./NLLoc/model/3km0.025.S.mod.*


In [ ]:
# Pg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Pg2tblanketPScindy_nll.in

# Sg2tblanketPScindy_nll.in
! ./src/Grid2Time ./PNR_run/Sg2tblanketPScindy_nll.in

Streaming output truncated to the last 5000 lines.
update side z=110: ff fb bb bf z=110 <R#1>updated.
update side z=109: ff fb bb bf z=109 <R#1>updated.
update side z=108: ff fb bb bf z=108 <R#1>updated.
update side z=107: ff fb bb bf z=107 <R#1>updated.
update side z=106: ff fb bb bf z=106 <R#1>updated.
update side z=105: ff fb bb bf z=105 <R#1>updated.
update side z=104: ff fb bb bf z=104 <R#1>updated.
update side z=103: ff fb bb bf z=103 <R#1>updated.
update side z=102: ff fb bb bf z=102 <R#1>updated.
update side z=101: ff fb bb bf z=101 <R#1>updated.
update side z=100: ff fb bb bf z=100 <R#1>updated.
update side z=99: ff fb bb bf z=99 <R#1>updated.
update side z=98: ff fb bb bf z=98 <R#1>updated.
update side z=97: ff fb bb bf z=97 <R#1>updated.
update side z=96: ff fb bb bf z=96 <R#1>updated.
update side z=95: ff fb bb bf z=95 <R#1>updated.
update side z=94: ff fb bb bf z=94 <R#1>updated.
update side z=93: ff fb bb bf z=93 <R#1>updated.
update side z=92: ff fb bb bf z=92 <R#1>updat

In [ ]:
import datetime

# NLLOC from code before
os.system('sed "s|LOCFILES ./NLLOC_OBS/PS_181015_NLLOC_OBS.hpf NLLOC_OBS ./NLLOC/time/3km0.025 ./NLLoc/loc/181015/ALLPS|LOCFILES ./PNR_run/testNLLOC_OBS.hpf NLLOC_OBS ./NLLoc/time/3km0.025 ./NLLoc/loc/ALLPS|g" "./PNR_run/blanketPScindy_nll.in" > "./PNR_run/blanketPScindy_best.in"')

start = datetime.datetime.now()
!./src/NLLoc ./PNR_run/blanketPScindy_best.in
end = datetime.datetime.now()
timing = end - start
print(timing)


Streaming output truncated to the last 5000 lines.
Arrival 4:  PR23 (PR23)  ? p ? 0  2018 12 11   09 59 43.886000  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 5:  PR23 (PR23)  ? s ? 0  2018 12 11   09 59 43.927500  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 6:  PR22 (PR22)  ? p ? 0  2018 12 11   09 59 43.890500  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 7:  PR22 (PR22)  ? s ? 0  2018 12 11   09 59 43.936000  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 8:  PR21 (PR21)  ? p ? 0  2018 12 11   09 59 43.896000  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 9:  PR21 (PR21)  ? s ? 0  2018 12 11   09 59 43.946000  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 10:  PR20 (PR20)  ? p ? 0  2018 12 11   09 59 43.901000  Unc: GAU 0.002500  Amp: -1.000000  Dur: -1.000000  Per: -1.000000
Arrival 11:  PR20 (PR20)  ? s ? 0  201

In [ ]:
##############################
# NLLOC output --> catalogue
##############################

# path to
origins = []
lats = []
lons = []
depths = []

with open('./NLLoc/loc/ALLPS.sum.grid0.loc.hyp','r') as fi:
  for ln in fi:
    if ln.startswith('GEOGRAPHIC'):
      origins.append(ln[15:42])
      lats.append(float(ln[47:57]))
      lons.append(float(ln[62:72]))
      depths.append(float(ln[78:]))

# loop through and convert origin times to UTCDateTime
for i in range(len(origins)):
  origins[i] = UTCDateTime(origins[i][:4] + origins[i][5:7] + origins[i][8:10] + 'T' + origins[i][12:14] + origins[i][15:17] + "%.4f" % float(origins[i][18:27].zfill(7)))

# create a dataframe of event locations
dfs = pd.DataFrame(list(zip(origins,lats,lons,depths)), columns = ['time', 'lat', 'lon', 'depth'])
# Define the wgs84 and osgb36 projection
wgs84 = pyproj.CRS("EPSG:4326")
osgb = pyproj.CRS("EPSG:27700")

lat = dfs['lat']
lon = dfs['lon']
xx, yy = pyproj.transform(wgs84,osgb,lat,lon)
dfs['xx'] = xx
dfs['yy'] = yy

# # uncomment to save
dfs.to_csv(drive+ '/Catalog/UGPD_PNR_catalogue.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [ ]:
dfs

,time,lat,lon,depth,xx,yy
0,2018-12-11T09:01:05.914900Z,53.787659,-2.961423,2.122656,336755.458402,432800.094040
1,2018-12-11T09:14:59.917600Z,53.787621,-2.963436,2.058203,336622.780907,432797.664378
2,2018-12-11T09:15:05.224600Z,53.789538,-2.965642,2.290234,336480.346722,433012.915097
3,2018-12-11T09:15:10.527900Z,53.787698,-2.966292,2.333203,336434.738891,432808.788477
4,2018-12-11T09:15:58.907100Z,53.787774,-2.966422,2.341797,336426.289376,432817.360452
...,...,...,...,...,...,...
1968,2018-12-11T09:59:57.542500Z,53.788771,-2.965772,2.350391,336470.621511,432927.698958
1969,2018-12-11T09:59:58.389000Z,53.788771,-2.966811,2.324609,336402.172127,432928.630715
1970,2018-12-11T09:59:59.107600Z,53.786432,-2.966098,2.277344,336445.603091,432667.765632
1971,2018-12-11T09:59:59.761000Z,53.788158,-2.966422,2.290234,336426.870995,432860.082394


___________